## Pivot points custom v8 Strategy

In [18]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pandas_ta as ta

In [19]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [31]:
FAST_EMA_LENGTH = 5
LOW_EMA_LENGTH = 26
def prepareData(htd):
    _1_d_df = htd.copy()
    _1_d_df['First_Open'] = _1_d_df['Open']
    _1_d_df['First_Close'] = _1_d_df['Close']
    _1_d_df['Second_Open'] = _1_d_df['Open']
    _1_d_df['Second_Close'] = _1_d_df['Close']
    _1_d_df = _1_d_df.resample("D").agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min',
        'Volume': 'sum',
        'First_Open': cal_first_open,
        'First_Close': cal_first_close,
        'Second_Open': cal_second_open,
        'Second_Close': cal_second_close
    })
    _1_d_df.dropna(inplace=True)
    _1_d_df = cal_pivots(_1_d_df)
    _1_d_df = _1_d_df[['P', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'First_Open',
                       'First_Close', 'Second_Open', 'Second_Close']]
    _1_d_df.dropna(inplace=True)

    htd = htd.assign(time_d=pd.PeriodIndex(htd.index, freq='1D').to_timestamp())
    htd = pd.merge(htd, _1_d_df, left_on="time_d", right_index=True, how="left")
    htd["current"] = htd.index + pd.DateOffset(minutes=5)
    htd['prev_Close'] = htd['Close'].shift(1)
    htd["ema_f"] = ta.ema(htd["Close"], length=FAST_EMA_LENGTH)
    htd["ema_f_shift"] = htd["ema_f"].shift(1)
    htd["ema_l"] = ta.ema(htd["Close"], length=LOW_EMA_LENGTH)
    htd["ema_l_shift"] = htd["ema_l"].shift(1)
    htd["ma_line"] = htd["Close"].rolling(20).mean()
    htd['above_ma'] = htd.apply(lambda r: 1 if r['Close'] > r['ma_line'] else 0, axis=1)
    htd['below_ma'] = htd.apply(lambda r: 1 if r['Close'] < r['ma_line'] else 0, axis=1)
    htd['total_above_ma'] = htd['above_ma'].rolling(150).sum()
    htd['total_below_ma'] = htd['below_ma'].rolling(150).sum()
    htd['trend'] = htd.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)
    htd.drop(columns=['ma_line', 'above_ma', 'below_ma', 'total_above_ma', 'total_below_ma'], inplace=True)
    htd.dropna(inplace=True)
    return htd


def cal_first_open(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute == 900]
    return tick


def cal_first_close(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute == 900]
    return tick


def cal_second_open(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute == 905]
    return tick


def cal_second_close(tick):
    tick = tick[100 * tick.index.hour + tick.index.minute == 905]
    return tick


def cal_pivot(row):
    pivot = (row['High_s'] + row['Low_s'] + row['Close_s']) / 3
    return pivot


def cal_r1(row):
    result = 2 * row['P'] - row['Low_s']
    return result


def cal_r2(row):
    result = row['P'] + row['High_s'] - row['Low_s']
    return result


def cal_r3(row):
    # result = row['P'] + 2 * (row['High_s'] - row['Low_s'])    # Classic
    result = row['P'] * 2 + row['High_s'] - 2 * row['Low_s']
    return result


def cal_r4(row):
    # result = row['P'] + 3 * (row['High_s'] - row['Low_s'])    # Classic
    result = row['P'] * 3 + row['High_s'] - 3 * row['Low_s']
    return result


def cal_r5(row):
    # result = row['P'] + 4 * (row['High_s'] - row['Low_s'])      # Classic
    result = row['P'] * 4 + row['High_s'] - 4 * row['Low_s']
    return result


def cal_r6(row):
    # result = row['P'] + 5 * (row['High_s'] - row['Low_s'])      # Classic
    result = row['P'] * 5 + row['High_s'] - 5 * row['Low_s']
    return result


def cal_s1(row):
    result = 2 * row['P'] - row['High_s']
    return result


def cal_s2(row):
    result = row['P'] - (row['High_s'] - row['Low_s'])
    return result


def cal_s3(row):
    # result = row['P'] - 2 * (row['High_s'] - row['Low_s'])  # Classic
    result = row['P'] * 2 - (2 * row['High_s'] - row['Low_s'])
    return result


def cal_s4(row):
    # result = row['P'] - 3 * (row['High_s'] - row['Low_s'])  # Classic
    result = row['P'] * 3 - (3 * row['High_s'] - row['Low_s'])
    return result


def cal_s5(row):
    # result = row['P'] - 4 * (row['High_s'] - row['Low_s'])  # Classic
    result = row['P'] * 4 - (4 * row['High_s'] - row['Low_s'])
    return result


def cal_s6(row):
    # result = row['P'] - 5 * (row['High_s'] - row['Low_s'])  # Classic
    result = row['P'] * 5 - (5 * row['High_s'] - row['Low_s'])
    return result


def cal_pivots(_1_d_df):
    _1_d_df['High_s'] = _1_d_df['High'].shift(1)
    _1_d_df['Low_s'] = _1_d_df['Low'].shift(1)
    _1_d_df['Close_s'] = _1_d_df['Close'].shift(1)
    _1_d_df['Height_s'] = _1_d_df['High_s'] - _1_d_df['Low_s']
    _1_d_df['Volume_s'] = _1_d_df['Volume'].shift(1)

    _1_d_df['P'] = _1_d_df.apply(
        lambda row: cal_pivot(row), axis=1)
    _1_d_df['R1'] = _1_d_df.apply(
        lambda row: cal_r1(row), axis=1)
    _1_d_df['R2'] = _1_d_df.apply(
        lambda row: cal_r2(row), axis=1)
    _1_d_df['R3'] = _1_d_df.apply(
        lambda row: cal_r3(row), axis=1)
    _1_d_df['R4'] = _1_d_df.apply(
        lambda row: cal_r4(row), axis=1)
    _1_d_df['R5'] = _1_d_df.apply(
        lambda row: cal_r5(row), axis=1)
    _1_d_df['R6'] = _1_d_df.apply(
        lambda row: cal_r6(row), axis=1)
    _1_d_df['S1'] = _1_d_df.apply(
        lambda row: cal_s1(row), axis=1)
    _1_d_df['S2'] = _1_d_df.apply(
        lambda row: cal_s2(row), axis=1)
    _1_d_df['S3'] = _1_d_df.apply(
        lambda row: cal_s3(row), axis=1)
    _1_d_df['S4'] = _1_d_df.apply(
        lambda row: cal_s4(row), axis=1)
    _1_d_df['S5'] = _1_d_df.apply(
        lambda row: cal_s5(row), axis=1)
    _1_d_df['S6'] = _1_d_df.apply(
        lambda row: cal_s6(row), axis=1)
    return _1_d_df

In [32]:
data = dataset.copy()
prepared_data = prepareData(data)

In [33]:
prepared_data.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'time_d', 'P', 'R1', 'R2',
       'R3', 'R4', 'R5', 'R6', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6',
       'First_Open', 'First_Close', 'Second_Open', 'Second_Close', 'current',
       'prev_Close', 'ema_f', 'ema_f_shift', 'ema_l', 'ema_l_shift', 'trend'],
      dtype='object')

## Strategy

In [34]:
def trade_simulation(prepared_data):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "Type", "Profit"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss at 20 pips
    sl_step = 3
    # Takeprofit at 60 pips(R/R = 1/3)
    tp_step = 9
    for i, row in prepared_data.iterrows():
            if has_open_deal:
                if type_open_deal == 'short':
                    if row['signal'] == 'close':
                        # close open deal at 2:25PM (dataframe)
                        profit = entry_price - row['Close']
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    # Stoploss
                    elif row['High'] > entry_price + sl_step:
                        profit = -sl_step
                        exit_price = entry_price + sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        """ Close short deal"""
                        body = abs(row['Close'] - row['Open'])
                        lower_shadow = min(row['Close'], row['Open']) - row['Low']
                        is_long_lower_shadow = True if lower_shadow > 2 else False
                        if body > 0.7 and is_long_lower_shadow and lower_shadow > 3 * body:
                            profit = entry_price - row['Close']
                            if profit < -sl_step:
                                profit = -sl_step
                            exit_price = row['Close']
                            exit_time = row.name
                            has_open_deal = False
                            type_open_deal = ''
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    #     # Takeprofit
                    #     if row['Low'] < entry_price - tp_step:
                    #         profit = tp_step
                    #         exit_price = entry_price - tp_step
                    #         exit_time = row.name
                    #         _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    #         has_open_deal = False
                    #         type_open_deal = ''
                elif type_open_deal == 'long':
                    if row['signal'] == 'close':
                        # close open deal at 2:25PM (dataframe)
                        profit = row['Close'] - entry_price
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                    # Stoploss
                    if row['Low'] < entry_price - sl_step:
                        profit = -sl_step
                        exit_price = entry_price - sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        """ Close long deal"""
                        body = abs(row['Close'] - row['Open'])
                        upper_shadow = row['High'] - max(row['Close'], row['Open'])
                        is_long_upper_shadow = True if upper_shadow > 2 else False
                        if body > 0.7 and is_long_upper_shadow and upper_shadow > 3 * body:
                            profit = row['Close'] - entry_price
                            if profit < -sl_step:
                                profit = -sl_step
                            exit_price = row['Close']
                            exit_time = row.name
                            has_open_deal = False
                            type_open_deal = ''
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                    #     # Takeprofit
                    #     if row['High'] > entry_price + tp_step:
                    #         profit = tp_step
                    #         exit_price = entry_price + tp_step
                    #         exit_time = row.name
                    #         _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                    #         has_open_deal = False
                    #         type_open_deal = ''
                        #
            if not has_open_deal:
                if row['signal'] == 'short':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'short'
                elif row['signal'] == 'long':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'long'
    return _trades

In [35]:
def first_2_candlesticks_same_color(signal_data):
    """
        9:05AM has same candlestick's color with 9:00AM
    """
    first_is_green = True if signal_data['First_Close'] > signal_data['First_Open'] else False
    first_is_doji = True if signal_data['First_Close'] == signal_data['First_Open'] else False
    second_is_green = True if signal_data['Second_Close'] > signal_data['Second_Open'] else False
    second_is_doji = True if signal_data['Second_Close'] == signal_data['Second_Open'] else False

    if first_is_green != second_is_green and not second_is_doji and not first_is_doji:
        return False
    else:
        return True

In [36]:
def open_signal_v3(signal_data):
    """ Copying from custom V3"""
    signal = ''
    if signal_data['prev_Close'] < signal_data['P'] < signal_data['Close']  and signal_data['trend'] == 'up':
        signal = 'long'
    elif signal_data['prev_Close'] < signal_data['R1'] < signal_data['Close'] \
            or signal_data['prev_Close'] < signal_data['R2'] < signal_data['Close'] or signal_data['prev_Close'] < \
            signal_data['R3'] < signal_data['Close'] \
            or signal_data['prev_Close'] < signal_data['R4'] < signal_data['Close']:
        if (0.5 < signal_data['Close'] - signal_data['P'] < 3.5) \
                or (0.5 < signal_data['Close'] - signal_data['R1'] < 3.5) \
                or (0.5 < signal_data['Close'] - signal_data['R2'] < 3.5) \
                or (0.5 < signal_data['Close'] - signal_data['R3'] < 3.5) \
                or (0.5 < signal_data['Close'] - signal_data['R4'] < 3.5):
            signal = 'long'
    if signal_data['prev_Close'] > signal_data['P'] > signal_data['Close']  and signal_data['trend'] == 'down':
        signal = 'short'
    elif signal_data['prev_Close'] > signal_data['S1'] > signal_data['Close'] \
            or signal_data['prev_Close'] > signal_data['S2'] > signal_data['Close'] \
            or signal_data['prev_Close'] > signal_data['S3'] > signal_data['Close'] \
            or signal_data['prev_Close'] > signal_data['S4'] > signal_data['Close']:
        if (0.5 < signal_data['P'] - signal_data['Close'] < 3.5) \
                or (0.5 < signal_data['S1'] - signal_data['Close'] < 3.5) \
                or (0.5 < signal_data['S2'] - signal_data['Close'] < 3.5) \
                or (0.5 < signal_data['S3'] - signal_data['Close'] < 3.5) \
                or (0.5 < signal_data['S4'] - signal_data['Close'] < 3.5):
            signal = 'short'
    return signal
#
def open_signal_version_omega(signal_data):
    signal = ''
    if signal_data['ema_f_shift'] < signal_data['ema_l_shift'] and signal_data['ema_f'] > signal_data['ema_l']:
        signal = 'long'
    elif signal_data['ema_f_shift'] > signal_data['ema_l_shift'] and signal_data['ema_f'] < signal_data['ema_l']:
        signal = 'short'
    return signal

In [37]:
def cal_signal(row):
    signal = ''
    if 915 < 100*row.name.hour + row.name.minute < 1420:
        if row['same_color'] == True:
            signal = open_signal_v3(row)
        else:
            signal = open_signal_version_omega(row)
    elif 100*row.name.hour + row.name.minute == 1425:
        signal = 'close'
    return signal

In [38]:
prepared_data['same_color'] = prepared_data.apply(lambda r: first_2_candlesticks_same_color(r), axis=1)
prepared_data['signal'] = prepared_data.apply(lambda r: cal_signal(r), axis=1)
trade_result = trade_simulation(prepared_data)
trade_result.Profit.sum()

1452.7000000000003

In [39]:
trade_result.Profit.count()

1759

In [40]:
trade_result.Profit.mean()

0.825866969869244

In [41]:
# win rate
len(trade_result[trade_result.Profit > 0]) / (len(trade_result[trade_result.Profit < 0]) + len(trade_result[trade_result.Profit > 0]))

0.35469107551487417

In [42]:
trade_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
0,2018-08-14 10:45:00,953.0,2018-08-14 13:15:00,956.0,Short,-3.0
1,2018-08-15 11:10:00,957.7,2018-08-15 14:25:00,951.2,Short,6.5
2,2018-08-16 09:25:00,940.8,2018-08-16 14:00:00,943.8,Short,-3.0
3,2018-08-17 10:55:00,948.8,2018-08-17 11:05:00,951.8,Short,-3.0
4,2018-08-17 11:05:00,952.1,2018-08-17 13:50:00,949.1,Long,-3.0
...,...,...,...,...,...,...
1754,2024-06-28 09:55:00,1287.2,2024-06-28 14:25:00,1277.9,Short,9.3
1755,2024-07-02 10:15:00,1290.8,2024-07-02 14:25:00,1296.6,Long,5.8
1756,2024-07-03 10:05:00,1295.5,2024-07-03 13:00:00,1298.5,Short,-3.0
1757,2024-07-04 13:20:00,1307.8,2024-07-04 14:25:00,1308.0,Short,-0.2


In [43]:
selected_month = trade_result[(trade_result.EntryTime > '2024-06-01 00:00:00') & (trade_result.EntryTime < '2024-06-30 15:00:00')]
selected_month.Profit.sum()

-34.90000000000009

In [46]:
selected_month

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
1731,2024-06-03 09:50:00,1283.9,2024-06-03 14:25:00,1293.7,Long,9.8
1732,2024-06-04 09:50:00,1300.8,2024-06-04 10:30:00,1297.8,Long,-3.0
1733,2024-06-05 11:00:00,1302.7,2024-06-05 13:15:00,1305.7,Short,-3.0
1734,2024-06-05 13:15:00,1304.2,2024-06-05 13:40:00,1301.2,Long,-3.0
1735,2024-06-05 13:55:00,1305.4,2024-06-05 14:15:00,1302.4,Long,-3.0
1736,2024-06-06 10:50:00,1300.4,2024-06-06 14:00:00,1297.4,Long,-3.0
1737,2024-06-06 14:10:00,1301.3,2024-06-06 14:20:00,1298.3,Long,-3.0
1738,2024-06-11 10:10:00,1310.2,2024-06-11 10:30:00,1307.2,Long,-3.0
1739,2024-06-12 11:15:00,1306.5,2024-06-12 13:15:00,1309.5,Short,-3.0
1740,2024-06-14 09:30:00,1330.4,2024-06-14 09:55:00,1333.4,Short,-3.0


In [44]:
from datetime import date
from datetime import timedelta
from datetime import datetime
last_expire_date = date.today() - timedelta(days=90)
last_expire_date = last_expire_date.strftime("%Y-%m-%d 00:00:00")
_last_3months = trade_result[trade_result.EntryTime > last_expire_date]
_last_3months.Profit.sum()

45.00000000000023

In [45]:
_last_3months

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
1694,2024-04-08 14:00:00,1255.7,2024-04-08 14:15:00,1258.7,Short,-3.0
1695,2024-04-09 13:15:00,1261.1,2024-04-09 14:25:00,1263.3,Long,2.2
1696,2024-04-10 09:40:00,1271.0,2024-04-10 13:20:00,1268.0,Long,-3.0
1697,2024-04-12 13:35:00,1269.7,2024-04-12 14:25:00,1282.3,Long,12.6
1698,2024-04-17 10:25:00,1226.4,2024-04-17 14:25:00,1215.0,Short,11.4
...,...,...,...,...,...,...
1754,2024-06-28 09:55:00,1287.2,2024-06-28 14:25:00,1277.9,Short,9.3
1755,2024-07-02 10:15:00,1290.8,2024-07-02 14:25:00,1296.6,Long,5.8
1756,2024-07-03 10:05:00,1295.5,2024-07-03 13:00:00,1298.5,Short,-3.0
1757,2024-07-04 13:20:00,1307.8,2024-07-04 14:25:00,1308.0,Short,-0.2
